<a href="https://colab.research.google.com/github/usaeva-a/PET-projects/blob/main/sportsman_error_prediction/recsys_test/skating_recsys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Рекомендательная система для школы по фигурному катанию</center></h1>

<font size="4"><b>Цель проекта</b></font>

В рамках данного проекта необходимо создать рекомендательную систему, которая будет помогать тренерам составлять программу тренировки и рекомендовать новые элементы, которые может выполнить фигурист.

В данном проекте будет продемонстрировано 2 рекомендательные системы:
1. На основе библиотеки Surprise (Simple Python RecommendatIon System Engine) - это Python-библиотека, разработанная для создания и оценки рекомендательных систем. Она будет подбирать элементы для каждого спортсмена с учётом базовых оценок за каждый элемент.
2. На основе коллаборативной фильтрации с использованием матричной факторизации, а именно Item-Item Collaborative Filtering, т.е. та которая опирается на схожесть между элементами. Т.е. если спортсмен выполняет определенный элемент, то, указав его, можно получить другие элементы, которые он также сможет попробовать выполнить.

<font size="4"><b>Исходные данные</b></font>

В наличии csv-файл, созданный мной на предыдущем этапе проекта.

Описание столбцов:

- unit_id: идентификатор юнита
- elements: название элемента фигурного катания (прыжок, вращение или дорожка шагов)
- bs_corrected: базовая оценка (идеал, цена данного элемента/комбинации, сложность), скорректированная на предыдущем этапе.


In [117]:
!pip install scikit-surprise

In [118]:
# импорт библиотек
import pandas as pd
import numpy as np
from collections import defaultdict

# загрузка инструментов для работы с библиотекой surprise
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

# загрузка модуля для работы с разреженными матрицами
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

# импортируем алгоритм k-ближайших соседей
from sklearn.neighbors import NearestNeighbors

## Загрузка данных

Используем таблицу, которую получили на предыдущем этапе. Загрузим её, просмотрим 10 случайных строчек и основную информацию методом info().

In [119]:
# сохранение данных для создания рекомендательной системы в переменной recsys
# и их предварительный просмотр
for_recsys = pd.read_csv('/content/sample_data/for_recsys.csv')
for_recsys.sample(10)

,unit_id,bs_corrected,elements
64971,1074,3.50,CCoSp4
147248,4502,0.50,1Lo
47503,1047,2.64,2A
115427,9453,1.10,1A
144212,86,2.70,LSp4
15694,455,5.30,3F
4861,626,7.60,4T
73477,1490,2.60,FCSSp3
60529,211,1.94,NaN
61746,184,0.50,2F


In [120]:
for_recsys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161342 entries, 0 to 161341
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   unit_id       161342 non-null  int64  
 1   bs_corrected  161342 non-null  float64
 2   elements      127159 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 3.7+ MB


Выполним небольшую предобработку данных:
- удалим пропуски в столбце `elements`, которые получились из-за того, что для построения рекомендательной системы исключили каскады.
- удалим строки-дубликаты в столбцах `unit_id`, `elements`, т.к. некоторые спортсмены выполняли элементы одного типа по несколько раз.
- удалим пустые строки в столбце `bs_corrected`, которые как раз соответствуют каскадам, которые мы не учитываем в данной работе.

In [121]:
# удаление пропусков
for_recsys = for_recsys.dropna()

In [122]:
# просмотр количества дублирующихся строчек
for_recsys.duplicated(subset=['unit_id', 'elements']).sum()

86928

In [123]:
# удаление дубликатов
for_recsys.drop_duplicates(subset=['unit_id', 'elements'], inplace=True)

<ipython-input-123-406590c93e68>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for_recsys.drop_duplicates(subset=['unit_id', 'elements'], inplace=True)


In [124]:
# удаление пустых строк в столбцах elements и bs_corrected
for_recsys = for_recsys.query('elements != "" and bs_corrected != ""')

## Рекомендательная система с использованием библиотеки Surprise

Для начала создадим рекомендательную систему на основе библиотеки Surprise. Она включает в себя определенные компоненты такие как Dataset, Reader для считывания, предобработки, настройки исходных данных, так и встроенные алгоритмы как, например, SVD, который мы будем использовать.

In [125]:
for_recsys['bs_corrected'].max()

11.5

In [126]:
# создаем объект Reader для определения формата данных
reader = Reader(rating_scale=(0, 11.5))

# создаем датасет из DataFrame и объекта Reader
dataset = Dataset.load_from_df(for_recsys[['unit_id', 'elements', 'bs_corrected']], reader)

In [127]:
trainset, testset = train_test_split(dataset, test_size=0.2, random_state=42)

In [128]:
# создание, обучение и получение предсказаний на модели SVD
model = SVD()
model.fit(trainset)
predictions = model.test(testset)

In [129]:
def get_top_n(predictions, n=10):
    """Возвращает топ-N лучших рекомендаций для каждого пользователя
    из набора прогнозов.

    Args:
        predictions(список предсказаний): Список прогнозов на тестовых данных.
        n(int): Количество рекомендаций для вывода для каждого пользователя.
        По умолчанию 10.

    Возвращает:
    Словарь, в котором ключами являются идентификаторы пользователей,
    а значениями — списки кортежей:
    [(идентификатор элемента, базовая оценка), ...] размера n.

    """

    # сначала сопоставим прогнозы с каждым пользователем
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # затем отсортируем прогнозы для каждого пользователя и
    # получим k самых высоких из них
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [130]:
# вызов функции для получения топ-10 рекомендаций
top_n = get_top_n(predictions, n=10)

In [131]:
def get_recommendations_for_unit(unit_id):
  """Возвращает топ-N лучших рекомендаций для конкретного
    спортсмена.
    Args:
        unit_id(int): идентификатор спортсмена, для которого нужно получить
        список рекомендаций
    Возвращает:
    Список элементов, рекомендуемых спортсмену.
  """
  data = top_n.get(unit_id)

  return list(map(lambda x: x[0], data))

In [133]:
recom_list = get_recommendations_for_unit(111)
print(f'Рекомендуется добавить элементы {recom_list}')

Рекомендуется добавить элементы ['CCoSp3', 'FSSp4', '2A', 'LSp4', 'FSSp3', 'FSSp1', 'StSq1']


## Коллаборативная фильтрация, основанная на сходстве элементов (Item-based)

Для построения этого типа рекомендательной системы необходимо создать матрицу взаимодействия спортсменов с элементами. В качестве столбцов будут названия элементов, в качестве строк - идентификационные номера спортсменов, а на пересечении будет фиксироваться факт взаимодействия: 1 - спортсмен выполнял данный элемент, 0 - не выполнял.

In [104]:
# добавим столбец, отображающий факт выполнения элемента фигуристом
for_recsys['done'] = 1

In [105]:
# построение матрицы взаимодействия
unit_element_matrix = for_recsys.pivot(columns = 'elements',
                                       index = 'unit_id',
                                       values= 'done'
                                       )
unit_element_matrix.fillna(0, inplace=True)

In [106]:
unit_element_matrix.head()

elements,1A,1F,1Lo,1Lz,1S,1T,2A,2F,2Lo,2Lz,...,SSp2,SSp3,SSp4,SSpB,StSq,StSq1,StSq2,StSq3,StSq4,USpB
unit_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
5,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0


In [107]:
unit_element_matrix.shape

(3020, 104)

Таким образом, получилась матрица взаимодействия между 3020 спортсменами и 104 элементами катания. Матрица получилась разреженная.

В данном случае для начала уменьшим размерность матрицы с помощью TruncatedSVD (Truncated Singular Value Decomposition). При использовании данного подхода будет уменьшаться количество столбцов. Для того, чтобы сохранить названия элементов, транспонируем матрицу unit_element_matrix.

In [108]:
# транспонируем матрицу, чтобы unit_id стали столбцами,
# а названия элементов - строками
X = unit_element_matrix.T
X.shape

(104, 3020)

In [109]:
X.head()

unit_id,1,3,4,5,6,7,8,9,10,11,...,34998,34999,35000,35001,35002,35003,35004,35005,35006,35024
elements,,,,,,,,,,,,,,,,,,,,,
1A,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1F,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1Lo,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1Lz,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1S,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
# снизим размерность матрицы с помощью TruncatedSVD
SVD = TruncatedSVD(n_components=10, random_state=17)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(104, 10)

Рассчитаем коэффициент корреляции Пирсона для преобразованной матрицы.

In [111]:
# расчёт матрицы коэффициентов корреляции Пирсона
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(104, 104)

Напишем функцию для получения списка элементов, которые может выполнить фигурист, выполняющий заданный нами элемент. Т.е. получим список элементов с коэффициентом корреляции выше 80% с интересующим нас элементом.

In [112]:
def recomended_elements(target_element):
  """
  функция для нахождения элементов, которые может выполнить фигурист,
  выполняющий заданный нами элемент.
  target_element : str
  """
  element_names = list(X.index)
  element = element_names.index(target_element)
  correlation_element = correlation_matrix[element]
  recommend = list(X.index[correlation_element > 0.80])

  # уберём из списка выбранный
  recommend.remove(target_element)
  return recommend[0:9]

In [113]:
# вызов функции для получения рекомендаций
recomendations = recomended_elements('3S')
print(f'Рекомендуется добавить элементы: {recomendations}')

Рекомендуется добавить элементы: ['2A', '3F', '3Lo', '3Lz', '3T', 'CCoSp4', 'CSp4', 'ChSq1', 'FCCoSp']


# Вывод

В ходе данного этапа работы были разработаны 2 рекомендательные системы:
1. На основе базовых оценок за элемент с помощью библиотеки Surprise.
2. На основе сходства элементов.